# This is an example notebook on how to use NAM library for shape matching
In this notebook we consider different refinement with the spectral embeddings


In [1]:
# import libraries
import sys 
sys.path.append('..')
from model.neural_adjoint_map import *
from model.optimizer import *
from methods.linear_ref import *
from methods.nam_ref import *


In [2]:
#load data
import yaml
import sys
sys.path.append('./../../NRP/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/')
import logging
from os import path as osp

from datasets import build_dataset
from models import build_model

import yaml
from utils.tensor_util import to_device,to_numpy
import torch
import numpy as np

#torch.cuda.set_device(0)

#pointcloud

DATASET='dt4d_interclass'

with open('./../../NRP/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/options/test/'+DATASET+'.yaml', 'r') as f:
    opt = yaml.load(f, Loader=yaml.SafeLoader)
opt['is_train']=False
test_set = build_dataset(opt['datasets']['test_dataset'])

shape1=test_set[3]['first']
shape2=test_set[3]['second']

In [ ]:

# Load configuration
DATASET='shrec19'
with open('./../../NRP/Unsupervised-Learning-of-Robust-Spectral-Shape-Matching/options/test/'+DATASET+'.yaml', 'r') as f:
    opt = yaml.load(f, Loader=yaml.SafeLoader)

test_set = build_dataset(opt['datasets']['test_dataset'])
opt['is_train'] = False
model = build_model(opt)
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize variables
n_exp = 50
n_k = len(range(20, 201, 20)) + 1
mse_nzo = np.zeros((n_exp, n_k))
g_nzo = np.zeros((n_exp, n_k))

#ind=np.random.randint(0,len(test_set),n_exp)
ind=np.arange(n_exp)

results = {}

zo = []
abzo =[]
ima_l = []
fsf_l = []

n_zo = []
n_ima = []
n_fsf = []

import h5py

for i in ind:
    data_x, data_y = to_device(test_set[i]['second'], device), to_device(test_set[i]['first'], device)
    if data_x['name'] != data_y['name']:
        dist_y = data_y['dist'].cpu()
        corr_x = data_x['corr']
        corr_y = data_y['corr']
        dist_y = to_numpy(dist_y)
        corr_x = to_numpy(corr_x)
        corr_y = to_numpy(corr_y)
        #C = torch.linalg.pinv(data_y['evecs'][corr_y, :20]) @ data_x['evecs'][corr_x, :20]
        #p2p_ini = nearest(data_x['evecs'][:, :20], data_y['evecs'][:, :20] @ C).cpu().numpy()
        with h5py.File('/home/ubuntu/NRP//NRP/nfr/results/mesh/'+DATASET+'/results.h5', 'r+') as hf:
            p2p_ini=hf[f'unsup_noref_{i}'][:]

        p2p=fsf(data_x['evecs'],data_y['evecs'],p2p_ini,20,201,20).cpu()
        fsf_l.append(model.metrics['geo_error'](dist_y, corr_y, corr_x, p2p, return_mean=False).mean().item())
        p2p=zoomout(data_x['evecs'],data_y['evecs'],p2p_ini,20,201,20).cpu()
        zo.append(model.metrics['geo_error'](dist_y, corr_y, corr_x, p2p, return_mean=False).mean().item())
        p2p=ima(data_x['evecs'],data_y['evecs'],p2p_ini,20,201,20).cpu()
        ima_l.append(model.metrics['geo_error'](dist_y, corr_y, corr_x, p2p, return_mean=False).mean().item())
        p2p=ab_zoomout(data_x['evecs'],data_y['evecs'],p2p_ini,20,201,20).cpu()
        abzo.append(model.metrics['geo_error'](dist_y, corr_y, corr_x, p2p, return_mean=False).mean().item())
        p2p=neural_ima(data_x['evecs'],data_y['evecs'],p2p_ini,20,201,20).cpu()
        n_ima.append(model.metrics['geo_error'](dist_y, corr_y, corr_x, p2p, return_mean=False).mean().item())
        p2p=neural_zoomout(data_x['evecs'],data_y['evecs'],p2p_ini,20,201,20).cpu()
        n_zo.append(model.metrics['geo_error'](dist_y, corr_y, corr_x, p2p, return_mean=False).mean().item())
        p2p=neural_fsf(data_x['evecs'],data_y['evecs'],p2p_ini,20,201,20).cpu()
        n_fsf.append(model.metrics['geo_error'](dist_y, corr_y, corr_x, p2p, return_mean=False).mean().item())

    i=i+1
    print(i)
print('fsf:',np.mean(fsf_l))
print('zo:',np.mean(zo))
print('ima:',np.mean(ima_l))
print('abzo:',np.mean(abzo))

print('n_fsf:',np.mean(n_fsf))
print('n_zo:',np.mean(n_zo))
print('n_ima:',np.mean(n_ima))



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
fsf: 0.037019912339746955
zo: 0.03625019071623683
ima: 0.07355247605592012
abzo: 0.036790350824594496
n_fsf: 0.05690628681331873
n_zo: 0.03595277238637209
n_ima: 0.058093071170151236


In [4]:
print('fsf:',np.mean(fsf_l[:len(n_fsf)]))
print('zo:',np.mean(zo[:len(n_fsf)]))
print('ima:',np.mean(ima_l[:len(n_fsf)]))
print('abzo:',np.mean(abzo[:len(n_fsf)]))

print('n_fsf:',np.mean(n_fsf[:len(n_fsf)]))
print('n_zo:',np.mean(n_zo[:len(n_fsf)]))
print('n_ima:',np.mean(n_ima[:len(n_fsf)]))


fsf: 0.037019912339746955
zo: 0.03625019071623683
ima: 0.07355247605592012
abzo: 0.036790350824594496
n_fsf: 0.05690628681331873
n_zo: 0.03595277238637209
n_ima: 0.058093071170151236
